In [11]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [4]:
hdfsclient = Client("amp-bdg-master.amplab.net", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/home/eecs/akmorrow/ADAM/endive"

In [5]:
executor_mem = "100g"
cores_per_executor = 8
num_executors = 50
EGR1_PATH = "hdfs://amp-spark-master.amp:8020/user/akmorrow/DREAMDATA/aggregated/icml_tfs/EGR1"

featurized = "hdfs://amp-spark-master.amp:8020/user/akmorrow/featurized/wavelets/EGR1_dnase_all_dim_256"

In [13]:
# this is where the magic happens
results = [] 
featureOutput = "/data/anv/featurized/EGR1_small"
res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           "fakePath",
                           logpath,
                           featureOutput,
                           seed=0,
                           cores_per_executor=cores_per_executor,
                           sample=1,
                           num_filters=512,
                           executor_mem=executor_mem,
                           num_partitions=cores_per_executor*num_executors,
                           num_executors=num_executors)
results.append(res)

{'kmerLength': 8, 'reference': '/home/eecs/akmorrow/ADAM/endive/workfiles/hg19.2bit', 'numPartitions': 2000, 'dnaseBams': '/data/anv/DREAMDATA/dnase_bams/merged_coverage/', 'saveTestPredictions': 'predictions/', 'aggregatedSequenceOutput': 'hdfs://amp-spark-master.amp:8020/user/akmorrow/DREAMDATA/aggregated/icml_tfs/EGR1', 'featuresOutput': '/data/anv/featurized/EGR1_dnase', 'seed': 0, 'dnaseNarrow': '/data/anv/DREAMDATA/DNASE/peaks/relaxed/', 'approxDim': 512, 'featurizeSample': 1, 'alphabetSize': 4, 'readFiltersFromDisk': False}


In [139]:
# chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
# chromosomes.append("chrX")
chromosomes=["chr15"]
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])

In [ ]:
results_df = cross_validate(featurized, 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            logpath,
                            numHoldOutChr=1, 
                            numHoldOutCell=1,
                            num_folds=1, 
                            negativeSamplingFreqs=[0.001],
                            mixtureWeights=[-1.0],
                            cores_per_executor=8,
                            num_executors=32,
                            regs=[1e-2])

In [ ]:
featuresPath = "/data/anv/featurized/EGR1_small"
predictionsOutput = "/data/anv/ROC/"
modelPath = ""
valCellTypes = string_to_enum_celltypes(['GM12878'])
results_df =run_solver_pipeline(featuresPath,
               logpath,
               hdfsclient,
               predictionsOutput, 
               modelPath,
               cores_per_executor=CORES_PER_EXECUTOR,
               num_executors=NUM_EXECUTORS,
               executor_mem=EXECUTOR_MEM,
               use_yarn=True,
               valChromosomes=chromosomess,
               valCellTypes=valCellTypes,
               reg=0.1,
               negativeSamplingFreq=1.0,
               predictedOutput=None,         
               valDuringSolve=True,
               base_config=BASE_KERNEL_PIPELINE_CONFIG)

FOLD 0
HOLDING OUT CHROMOSOMES ['chr15']
HOLDING OUT CELL TYPES [3]
RUNING SOLVER WITH REG=0.01
AND MIXTUREWEIGHT=-1.0


In [ ]:
results_df